In [1]:
import pandas as pd
import numpy as np
import duckdb

In [2]:
data_folder = "staticNotDeployed/"
static_folder = "static/"

In [43]:
# scenario_BNZ_path = "simulations_new/BNZ.csv"
# scenario_holder_path = "simulations_new/BNZ.csv"

# New data march 2025
scenario_BNZ_path = "simulations_new/march2025/BNZ_£millions_annualy.csv"
scenario_holder_path = "simulations_new/march2025/BNZ_£millions_annualy.csv"

socio_factors_path = "UK_Archetypes_global_measures.csv"

all_scenarios = [scenario_BNZ_path, scenario_holder_path]
scenario_names = ["BNZ", "test"]

## Create pandas tables

In [44]:
df_socio = pd.read_csv(static_folder+socio_factors_path)

/var/folders/k6/5lxskccn2055kqvnfry46clc0000gr/T/ipykernel_94873/3418573967.py:1: DtypeWarning: Columns (7,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_socio = pd.read_csv(static_folder+socio_factors_path)


In [45]:
# Remove one row on NaN
df_socio = df_socio.dropna()

In [46]:
df_socio = df_socio.convert_dtypes()

In [47]:
df_socio

,Nation,LAD,MSOA,LSOA.DZ.CD,LSOA.DZ.NM,Under.35,Over.65,EPC,Median.Income,Tenure,...,Rurality,House.value,Fuel.Type,Fuel.consumption.total,Floor.area,Gas.flag,Number.cars,Urban.trips,Total.vkm,Urban.vkm
0,NI,N09000001,-,N20000001,Dunsilly_A1,0.75,0.25,4,23318,1,...,2,137688,3,37665,180,0,2,414,12923,2584
1,NI,N09000001,-,N20000002,Dunsilly_B1,0.65,0.35,4,23318,1,...,2,149950,3,35381,167,0,2,376,11730,2346
2,NI,N09000001,-,N20000003,Dunsilly_A2,0.66,0.34,4,23318,1,...,2,137688,3,32461,143,0,2,369,11504,2301
3,NI,N09000001,-,N20000004,Dunsilly_A3,0.73,0.27,5,23318,1,...,2,137688,3,30303,126,0,2,387,12075,2415
4,NI,N09000001,-,N20000005,Dunsilly_B2,0.7,0.3,4,23318,1,...,2,134250,3,35166,182,0,2,408,12719,2544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46547,Scotland,S12000040,-,S01013477,Broxburn South - 06,0.34,0.23,3.0,23951,1,...,1,175000,1,31622,97,1.0,2,392,15827,10491
46548,Scotland,S12000040,-,S01013478,Broxburn East - 01,0.42,0.14,3.0,34382,1,...,1,130000,1,12784,68,1.0,2,699,15224,10091
46549,Scotland,S12000040,-,S01013479,Broxburn East - 02,0.34,0.26,3.0,21767,1,...,1,72500,1,15801,69,1.0,2,461,14259,9452
46550,Scotland,S12000040,-,S01013480,Broxburn East - 03,0.4,0.23,3.0,23972,1,...,1,72500,1,17025,75,1.0,2,360,13406,8886


In [48]:
df_socio.rename(columns=lambda x: x.replace('.', '_'), inplace=True)

In [49]:
df_socio.columns

Index(['Nation', 'LAD', 'MSOA', 'LSOA_DZ_CD', 'LSOA_DZ_NM', 'Under_35',
       'Over_65', 'EPC', 'Median_Income', 'Tenure', 'Typology', 'Unemployment',
       'Rurality', 'House_value', 'Fuel_Type', 'Fuel_consumption_total',
       'Floor_area', 'Gas_flag', 'Number_cars', 'Urban_trips', 'Total_vkm',
       'Urban_vkm'],
      dtype='object')

In [50]:
# EPC and gas_flag have weird values
df_socio.dtypes

Nation                    string[python]
LAD                       string[python]
MSOA                      string[python]
LSOA_DZ_CD                string[python]
LSOA_DZ_NM                string[python]
Under_35                         Float64
Over_65                          Float64
EPC                               object
Median_Income                      Int64
Tenure                             Int64
Typology                           Int64
Unemployment                     Float64
Rurality                           Int64
House_value                        Int64
Fuel_Type                          Int64
Fuel_consumption_total             Int64
Floor_area                         Int64
Gas_flag                          object
Number_cars                        Int64
Urban_trips                        Int64
Total_vkm                          Int64
Urban_vkm                          Int64
dtype: object

In [51]:
set(df_socio.EPC)

{1.0, '2', 2.0, '3', 3.0, '4', 4.0, '5', 5.0, '6', 6.0, '7', 7.0, 'd'}

In [52]:
set(df_socio.Gas_flag)

{'0', 0.0, '1', 1.0, 'Y'}

In [53]:
# Replace the string 'unknown' with pd.NA
df_socio['Gas_flag'] = df_socio['Gas_flag'].replace('Y', pd.NA)

# Step 2: Convert to numeric (converts '1.0' to 1.0 as float)
df_socio['Gas_flag'] = pd.to_numeric(df_socio['Gas_flag'], errors='coerce')

df_socio['Gas_flag'] = df_socio['Gas_flag'].astype('Int16')

In [54]:
# Replace the string 'unknown' with pd.NA
df_socio['EPC'] = df_socio['EPC'].replace('d', pd.NA)

# Step 2: Convert to numeric (converts '1.0' to 1.0 as float)
df_socio['EPC'] = pd.to_numeric(df_socio['EPC'], errors='coerce')

df_socio['EPC'] = df_socio['EPC'].astype('Int16')

### Cobenefit table

In [55]:
dfs = []
for i, scenario in enumerate(all_scenarios):
    df_one_scenario = pd.read_csv(data_folder + scenario)
    df_one_scenario["scenario"] = scenario_names[i]
    dfs.append(df_one_scenario)
df = pd.concat(dfs, axis=0)
del dfs 

/var/folders/k6/5lxskccn2055kqvnfry46clc0000gr/T/ipykernel_94873/4264617398.py:3: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_one_scenario = pd.read_csv(data_folder + scenario)
/var/folders/k6/5lxskccn2055kqvnfry46clc0000gr/T/ipykernel_94873/4264617398.py:3: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_one_scenario = pd.read_csv(data_folder + scenario)


In [56]:
df

,Lookup Value,Coben,2025,2026,2027,2028,2029,2030,2031,2032,...,2042,2043,2044,2045,2046,2047,2048,2049,2050,scenario
0,N20000001,Air quality,0.003495246,0.004545631,0.005846987,0.007227724,0.008178627,0.008060945,0.008426776,0.00894241,...,0.010348311,0.010452822,0.010551088,0.010555385,0.010572823,0.010527148,0.010476093,0.010388816,0.010267653,BNZ
1,N20000001,Noise,0.001345039,0.002122763,0.003191027,0.003107497,0.003972673,0.006072646,0.006876487,0.006949075,...,0.010073978,0.010399989,0.010247743,0.010086102,0.009847787,0.009746151,0.009516318,0.009292782,0.009160228,BNZ
2,N20000001,Congestion,0.002332261,0.002217988,0.002029615,0.001955506,0.001207442,0.000834605,0.001329813,0.001140636,...,0.005494149,0.006463912,0.007158383,0.009144035,0.009932181,0.010756816,0.011624006,0.012209335,0.015007448,BNZ
3,N20000001,Road repairs,0.00273376,0.002985896,0.003331585,0.003500383,0.003506608,0.003967531,0.00394061,0.003570292,...,0.00290232,0.002680539,0.002530137,0.002735917,0.002624259,0.002645503,0.002454615,0.002443885,0.00250327,BNZ
4,N20000001,Road safety,0.000656373,0.000483755,0.000399042,0.000309456,2.17259E-05,-8.8596E-05,-0.000157228,-0.000281346,...,0.001681856,0.002021088,0.002386878,0.003245899,0.00370926,0.004182214,0.004577417,0.00500009,0.0058698,BNZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557107,S01013481,Excess cold,0.000104,0.00121,0.000733,0.000855,0.001428,0.004499,0.00249,0.002791,...,0.008979,0.00983,0.011646,0.012001,0.011921,0.012764,0.012686,0.012558,0.012818,test
557108,S01013481,Excess heat,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
557109,S01013481,Dampness,0.0,0.000004,0.000037,0.000056,0.000066,0.000161,0.000109,0.000111,...,0.000254,0.00027,0.000291,0.000282,0.00028,0.000278,0.000276,0.00027,0.000271,test
557110,S01013481,Diet change,0.019516,0.032003,0.04224,0.050937,0.033714,0.025009,0.028114,0.022994,...,0.020149,0.019883,0.023368,0.023059,0.027021,0.018846,0.018592,0.021738,0.014336,test


In [57]:
# Step: Drop rows where any column contains the value '#DIV/0!'
# There are rows with these weird values in the new dataset
df = df[~df.isin(['#DIV/0!']).any(axis=1)]

In [58]:
df.dtypes

Lookup Value    object
Coben           object
2025            object
2026            object
2027            object
2028            object
2029            object
2030            object
2031            object
2032            object
2033            object
2034            object
2035            object
2036            object
2037            object
2038            object
2039            object
2040            object
2041            object
2042            object
2043            object
2044            object
2045            object
2046            object
2047            object
2048            object
2049            object
2050            object
scenario        object
dtype: object

In [59]:
# Convert columns with names from 2025 to 2050 to float
year_columns = [str(year) for year in range(2025, 2051)]
df[year_columns] = df[year_columns].astype(np.float32)

/var/folders/k6/5lxskccn2055kqvnfry46clc0000gr/T/ipykernel_94873/2901450854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[year_columns] = df[year_columns].astype(np.float32)


In [60]:
df.dtypes

Lookup Value     object
Coben            object
2025            float32
2026            float32
2027            float32
2028            float32
2029            float32
2030            float32
2031            float32
2032            float32
2033            float32
2034            float32
2035            float32
2036            float32
2037            float32
2038            float32
2039            float32
2040            float32
2041            float32
2042            float32
2043            float32
2044            float32
2045            float32
2046            float32
2047            float32
2048            float32
2049            float32
2050            float32
scenario         object
dtype: object

In [61]:
# Create total column
# df["total (£m)"] = df[ [ f'{i} (£m)' for i in range(2025, 2051)]].sum(axis=1)
df["total (£m)"] = df[ [ f'{i}' for i in range(2025, 2051)]].sum(axis=1)

/var/folders/k6/5lxskccn2055kqvnfry46clc0000gr/T/ipykernel_94873/327882055.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["total (£m)"] = df[ [ f'{i}' for i in range(2025, 2051)]].sum(axis=1)


In [62]:
# Delete all the columns per hh, only keep total value columns
# Not needed with new data
# df = df.drop(columns=[ f'{i} (£/hh)' for i in range(2025, 2051)])

In [63]:
# Rename columns so it does not contain special characters (not needed anymore I think)
df.columns = df.columns.str.replace(' (£m)', '')

In [64]:
# Rename columns: replace spaces with underscores
df.columns = df.columns.str.replace(' ', '_')

In [65]:
df.head()

,Lookup_Value,Coben,2025,2026,2027,2028,2029,2030,2031,2032,...,2043,2044,2045,2046,2047,2048,2049,2050,scenario,total
0,N20000001,Air quality,0.003495,0.004546,0.005847,0.007228,0.008179,0.008061,0.008427,0.008942,...,0.010453,0.010551,0.010555,0.010573,0.010527,0.010476,0.010389,0.010268,BNZ,0.237583
1,N20000001,Noise,0.001345,0.002123,0.003191,0.003107,0.003973,0.006073,0.006876,0.006949,...,0.010400,0.010248,0.010086,0.009848,0.009746,0.009516,0.009293,0.009160,BNZ,0.207463
2,N20000001,Congestion,0.002332,0.002218,0.002030,0.001956,0.001207,0.000835,0.001330,0.001141,...,0.006464,0.007158,0.009144,0.009932,0.010757,0.011624,0.012209,0.015007,BNZ,0.122029
3,N20000001,Road repairs,0.002734,0.002986,0.003332,0.003500,0.003507,0.003968,0.003941,0.003570,...,0.002681,0.002530,0.002736,0.002624,0.002646,0.002455,0.002444,0.002503,BNZ,0.079201
4,N20000001,Road safety,0.000656,0.000484,0.000399,0.000309,0.000022,-0.000089,-0.000157,-0.000281,...,0.002021,0.002387,0.003246,0.003709,0.004182,0.004577,0.005000,0.005870,BNZ,0.036578


In [66]:
# df.astype({'total': 'float'})

In [67]:
df.total

0         2.375832e-01
1         2.074626e-01
2         1.220294e-01
3         7.920101e-02
4         3.657842e-02
              ...     
557107    1.804824e-01
557108    9.130131e-07
557109    4.796808e-03
557110    6.356870e-01
557111    3.583661e+00
Name: total, Length: 1109282, dtype: float32

In [68]:
np.min(df.total)

-6.758290767669678

In [69]:
np.max(df.total)

149.6426239013672

In [70]:
df.dtypes

Lookup_Value     object
Coben            object
2025            float32
2026            float32
2027            float32
2028            float32
2029            float32
2030            float32
2031            float32
2032            float32
2033            float32
2034            float32
2035            float32
2036            float32
2037            float32
2038            float32
2039            float32
2040            float32
2041            float32
2042            float32
2043            float32
2044            float32
2045            float32
2046            float32
2047            float32
2048            float32
2049            float32
2050            float32
scenario         object
total           float32
dtype: object

## Join SE factors and cobenefs

In [71]:
df = pd.merge(df, df_socio, left_on='Lookup_Value', right_on='LSOA_DZ_CD', how='inner')

In [72]:
df.head()

,Lookup_Value,Coben,2025,2026,2027,2028,2029,2030,2031,2032,...,Rurality,House_value,Fuel_Type,Fuel_consumption_total,Floor_area,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm
0,N20000001,Air quality,0.003495,0.004546,0.005847,0.007228,0.008179,0.008061,0.008427,0.008942,...,2,137688,3,37665,180,0,2,414,12923,2584
1,N20000001,Noise,0.001345,0.002123,0.003191,0.003107,0.003973,0.006073,0.006876,0.006949,...,2,137688,3,37665,180,0,2,414,12923,2584
2,N20000001,Congestion,0.002332,0.002218,0.002030,0.001956,0.001207,0.000835,0.001330,0.001141,...,2,137688,3,37665,180,0,2,414,12923,2584
3,N20000001,Road repairs,0.002734,0.002986,0.003332,0.003500,0.003507,0.003968,0.003941,0.003570,...,2,137688,3,37665,180,0,2,414,12923,2584
4,N20000001,Road safety,0.000656,0.000484,0.000399,0.000309,0.000022,-0.000089,-0.000157,-0.000281,...,2,137688,3,37665,180,0,2,414,12923,2584


In [73]:
# Rename cobenef to always keep the same name (changed depending the version)
df.rename(columns={'Coben': 'co_benefit_type'}, inplace=True)

## Aggregating the time

In [74]:
# Number of years merging
time_step = 5

In [75]:
years = list(range( 2025, 2051 ))
#years

In [76]:
df[["2025", "2026"]]

,2025,2026
0,3.495246e-03,4.545631e-03
1,1.345039e-03,2.122763e-03
2,2.332261e-03,2.217988e-03
3,2.733760e-03,2.985896e-03
4,6.563730e-04,4.837550e-04
...,...,...
1112239,1.039560e-04,1.210047e-03
1112240,7.523670e-10,-1.061550e-08
1112241,6.166430e-08,3.779780e-06
1112242,1.951576e-02,3.200329e-02


In [77]:
for i in range(0, len(years) - ( time_step - 1), time_step):
    window_years = [str(year) for year in years[i:i+5]]
    print(window_years)
    window_sum = df[window_years].sum(axis=1)
    df[f'Y{window_years[0]}_{window_years[-1]}'] = window_sum

['2025', '2026', '2027', '2028', '2029']
['2030', '2031', '2032', '2033', '2034']
['2035', '2036', '2037', '2038', '2039']
['2040', '2041', '2042', '2043', '2044']
['2045', '2046', '2047', '2048', '2049']


In [79]:
df[ ['2025', '2026', '2027', '2028', '2029', 'Y2025_2029'] ]

,2025,2026,2027,2028,2029,Y2025_2029
0,3.495246e-03,4.545631e-03,5.846987e-03,7.227724e-03,8.178627e-03,2.929422e-02
1,1.345039e-03,2.122763e-03,3.191027e-03,3.107497e-03,3.972673e-03,1.373900e-02
2,2.332261e-03,2.217988e-03,2.029615e-03,1.955506e-03,1.207442e-03,9.742812e-03
3,2.733760e-03,2.985896e-03,3.331585e-03,3.500383e-03,3.506608e-03,1.605823e-02
4,6.563730e-04,4.837550e-04,3.990420e-04,3.094560e-04,2.172590e-05,1.870352e-03
...,...,...,...,...,...,...
1112239,1.039560e-04,1.210047e-03,7.329670e-04,8.553590e-04,1.427739e-03,4.330068e-03
1112240,7.523670e-10,-1.061550e-08,-9.149760e-09,-2.776760e-08,-2.545340e-08,-7.223389e-08
1112241,6.166430e-08,3.779780e-06,3.685530e-05,5.593680e-05,6.588840e-05,1.625220e-04
1112242,1.951576e-02,3.200329e-02,4.223969e-02,5.093737e-02,3.371386e-02,1.784100e-01


In [80]:
# Delete single values columns for space
df = df.drop(columns=[str(year) for year in years])

In [81]:
# CONVERT INT64 into int32
df[df.select_dtypes(np.int64).columns] = df.select_dtypes(np.int64).astype(np.int32)

In [82]:
df

,Lookup_Value,co_benefit_type,scenario,total,Nation,LAD,MSOA,LSOA_DZ_CD,LSOA_DZ_NM,Under_35,...,Gas_flag,Number_cars,Urban_trips,Total_vkm,Urban_vkm,Y2025_2029,Y2030_2034,Y2035_2039,Y2040_2044,Y2045_2049
0,N20000001,Air quality,BNZ,2.375832e-01,NI,N09000001,-,N20000001,Dunsilly_A1,0.75,...,0,2,414,12923,2584,2.929422e-02,4.477071e-02,4.903598e-02,5.169436e-02,5.252027e-02
1,N20000001,Noise,BNZ,2.074626e-01,NI,N09000001,-,N20000001,Dunsilly_A1,0.75,...,0,2,414,12923,2584,1.373900e-02,3.627460e-02,4.907896e-02,5.072070e-02,4.848914e-02
2,N20000001,Congestion,BNZ,1.220294e-01,NI,N09000001,-,N20000001,Dunsilly_A1,0.75,...,0,2,414,12923,2584,9.742812e-03,5.053860e-03,1.022713e-02,2.833174e-02,5.366638e-02
3,N20000001,Road repairs,BNZ,7.920101e-02,NI,N09000001,-,N20000001,Dunsilly_A1,0.75,...,0,2,414,12923,2584,1.605823e-02,1.818745e-02,1.550715e-02,1.404074e-02,1.290418e-02
4,N20000001,Road safety,BNZ,3.657842e-02,NI,N09000001,-,N20000001,Dunsilly_A1,0.75,...,0,2,414,12923,2584,1.870352e-03,-1.044911e-03,7.986065e-04,8.369693e-03,2.071488e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112239,S01013481,Excess cold,test,1.804824e-01,Scotland,S12000040,-,S01013481,Broxburn East - 04,0.45,...,1,2,298,18643,12357,4.330068e-03,1.975439e-02,3.440068e-02,4.724859e-02,6.193027e-02
1112240,S01013481,Excess heat,test,9.130131e-07,Scotland,S12000040,-,S01013481,Broxburn East - 04,0.45,...,1,2,298,18643,12357,-7.223389e-08,-1.200843e-07,1.415980e-07,3.804091e-07,4.100572e-07
1112241,S01013481,Dampness,test,4.796808e-03,Scotland,S12000040,-,S01013481,Broxburn East - 04,0.45,...,1,2,298,18643,12357,1.625220e-04,6.381780e-04,1.083056e-03,1.257128e-03,1.385116e-03
1112242,S01013481,Diet change,test,6.356870e-01,Scotland,S12000040,-,S01013481,Broxburn East - 04,0.45,...,1,2,298,18643,12357,1.784100e-01,1.165155e-01,1.083907e-01,1.087795e-01,1.092549e-01


In [83]:
set(df.EPC)

{1, 2, 3, 4, 5, 6, 7, <NA>}

In [84]:
df.dtypes

Lookup_Value                      object
co_benefit_type                   object
scenario                          object
total                            float32
Nation                    string[python]
LAD                       string[python]
MSOA                      string[python]
LSOA_DZ_CD                string[python]
LSOA_DZ_NM                string[python]
Under_35                         Float64
Over_65                          Float64
EPC                                Int16
Median_Income                      int32
Tenure                             int32
Typology                           int32
Unemployment                     Float64
Rurality                           int32
House_value                        int32
Fuel_Type                          int32
Fuel_consumption_total             int32
Floor_area                         int32
Gas_flag                           Int16
Number_cars                        int32
Urban_trips                        int32
Total_vkm       

## Export table as parquet file

In [85]:
df.to_parquet('static/database.parquet')

In [35]:
# Not needed anymore, everything is in the same table
# df_socio.to_parquet('static/tableSocio.parquet')

## Create Duckdb instance

The parquet file is currently used in the frontend as the .duckdb file was causing error. This part is therefore not useful for now

In [42]:
DB_FILE_PATH = 'static/database.duckdb'
TABLE_NAME = "cobenefits"

In [43]:
con = duckdb.connect(DB_FILE_PATH)

In [46]:
# Create table and insert data
con.execute(f"DROP TABLE {TABLE_NAME}")

# Create table and insert data
con.execute(f"CREATE TABLE {TABLE_NAME} AS SELECT * FROM df")

# Verify data
result = con.execute(f"SELECT * FROM {TABLE_NAME} LIMIT 5").fetchall()
print("Sample data:")
print(result)

# Get and print schema
schema = con.execute(f"DESCRIBE {TABLE_NAME}").fetchall()
print("\nTable schema:")
for column in schema:
    print(f"{column[0]}: {column[1]}")

print(f"\nDatabase created and saved to: {DB_FILE_PATH}")

Sample data:
[('N20002754', 'Air quality', 138, 0.001385182, 0.001690344, 0.002178994, 0.00265043, 0.002930518, 0.002957105, 0.003004358, 0.003154786, 0.003284162, 0.003392733, 0.003309062, 0.003258978, 0.003316478, 0.003358668, 0.00338563, 0.003360311, 0.003329033, 0.00331858, 0.003300776, 0.003273695, 0.003256345, 0.003240189, 0.00319424, 0.003143862, 0.003091112, 0.00303487, 'BNZ', 0.078800441), ('N20002754', 'Noise', 138, 0.000150076, 0.000145001, 0.000140098, 0.00013536, 0.000774951, 0.001122295, 0.001103271, 0.001084611, 0.001066307, 0.001048349, 0.001038151, 0.001020742, 0.001003659, 0.000986897, 0.000970447, 0.001206611, 0.001182306, 0.001158578, 0.001135413, 0.001112795, 0.001176065, 0.001151699, 0.00112793, 0.00110474, 0.001082113, 0.00106603, 'BNZ', 0.024294494999999996), ('N20002754', 'Congestion', 138, 0.001159192, 0.001048197, 0.000942343, 0.000892502, 0.000496322, 0.000302559, 0.000561585, 0.000513852, 0.000433189, 0.000542365, 0.001034313, 0.001105454, 0.001294352, 0.00